In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import re
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools


Schwab_BASE = 'https://www.schwabjobs.com'
PNC_BASE = 'https://www.indeed.com/viewjob?'

PNC_URL = "https://www.indeed.com/jobs?q=Pnc+Bank"
Schwab_URL = "https://www.schwabjobs.com/List/Custom/Schwab-Job-Category"

stateList = []

banks = ["Schwab"]
bank = ''
listId = 1
count = 1
BASE = 'https://www.indeed.com/'
FIRST_LISTING_PAGE = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?clearPrefilter=1'
LISTING_URL_PREFIX = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start='
LISTING_URL_POSTFIX = 'clearPrefilter=1#cmp-menu-container'

def keywordsFileName(listId):
    if listId == 1:
        return 'frequency.csv'
    elif listId == 2:
        return 'tfIdfWords.csv'
    else:
        return 'tfIdfWords.csv'

def scrape(postingUrl):
    print(postingUrl)
    global count
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]

    values = [count, bank, listId, postingUrl, listId]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open(keywordsFileName(listId), 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        if bank == 'PNC Bank':
            value = soup2.find('input', {'id':'preLoadJSON'})
            for word in value.get('value').split('"CandLandPageText"')[1].replace("\\u003", " ").split():
                if(word in arr):
                    ind = arr.index(word)
                    wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            dataDict = postingDict.copy()  
            return {**postingDict, **wordCountDict}
                
        else:
            value = soup2.findAll("div", {"class": "desc"})
            for val in value:
                st = val.text.split()
                for word in st:
                    if(word in arr):
                        ind = arr.index(word)
                        wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    except:
        return ("An exception occurred")
    
    count = count + 1

    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 
 
def scrape_page_pnc(page):
    response = urlopen(page)
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        links = bs_obj.findAll('div',{'class': 'cmp-section cmp-with-border'})
        for page in links:
            linkTags = page.find_all('a', attrs={'href': re.compile("/cmp/_")})
            for tag in linkTags:
                link = tag.get('href')
                postingUrl = BASE + link
                pageDict = scrape(postingUrl)
                if (type(pageDict) is dict):
                        data = pd.DataFrame([pageDict])
                        # if file does not exist write header 
                        if not os.path.isfile('pncData.csv'):
                            data.to_csv('pncData.csv', header='column_names')
                        else: 
                            # else it exists so append without writing the header
                            data.to_csv('pncData.csv', mode='a', header=False)
                else:
                    print("exception?")
    


def scrape_pages_schwab(pages):
    stateList = {}
    for p in pages:
        pageHtml = urlopen(p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = Schwab_BASE + link.get('href')
            state = table.findAll("td",{"class": "colstate"})
            for s in state:
                st = re.sub(r'[^ \w\.]', '', s.text)
                if st not in stateList:
                    stateList[st] = 1
                else:
                    stateList[st] += 1
            pageDict = scrape(page)
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('data.csv'):
                    data.to_csv('data.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('data.csv', mode='a', header=False)
    with open('states.csv', 'a+', encoding = 'utf-8') as f: 
        for key in stateList.keys():
            f.write("%s,%d\n"%(key,stateList[key]))
        

def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(Schwab_BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(Schwab_BASE + key)
    return pageLinks

def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []
        cat = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
                cat.append(tag.text)
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        #categCsv = dict(zip(cat, numPages))
        #plt.bar(range(len(categCsv)), list(categCsv.values()), align='center')
        #plt.xticks(range(len(categCsv)), list(categCsv.keys()))
        #plt.xticks(rotation = 90)
        #plt.imshow()
        return dict(zip(catLinks, numPages))

def scrape_site(Url):
    scrape_page_pnc(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        scrape_page_pnc(nextPageUrl)

def get_data ():
    if (bank == "PNC"):
        scrape_site(PNC_URL)
    elif (bank == "Schwab"):
        res = urllib2.urlopen(Schwab_URL)
        categories = get_category(res)
        pages = get_pages(categories)
        scrape_pages_schwab(pages)

if __name__ == '__main__':
    if os.path.isfile('data.csv'):
        os.remove('data.csv')
    for i in range(1,2):
        listId = i
        print(listId)
        for b in banks:
            bank = b
            print(bank)
            get_data()


1
Schwab


NameError: name 'urlopen' is not defined

In [46]:
! pip install itertools

  Could not find a version that satisfies the requirement itertools (from versions: )
No matching distribution found for itertools
